In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
from keras.models import Model
from keras import backend as K
from utils.helper import heatmap
from innvestigate import utils as iutils
import os
from keras.preprocessing.image import img_to_array, load_img
from utils.fileutils import CacheStorage

os.environ["CUDA_VISIBLE_DEVICES"]="1"

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from utils.evaluations import get_validation_dataflow
from utils.evaluations import get_validation_data, ImageNetPreprocessGenerator
from utils.evaluations import get_averaged_validation_image, get_saliency_maps, get_predkeeper

In [4]:
valid_data_dir = Path("./data/ILSVRC2012_val/img_centercrop/")
val_imgs, val_labels = get_validation_data(valid_data_dir)
print(val_imgs.shape)
print(val_labels.shape)

Try to load cache file
(50000, 224, 224, 3)
(50000, 1000)


In [5]:
model = VGG19(weights="imagenet")
model.compile(
    loss='categorical_crossentropy',
    optimizer='SGD',
    metrics=["accuracy", "top_k_categorical_accuracy"],
)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
partial_model = Model(
    inputs=model.inputs,
    outputs=iutils.keras.graph.pre_softmax_tensors(model.outputs),
    name=model.name,
)

In [8]:
partial_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
input_imgs = val_imgs #preprocess_input(val_imgs) #for built in keras models

In [10]:
min_input = np.min(val_imgs)
max_input = np.max(val_imgs)
input_shape = val_imgs.shape
num_classes = val_labels.shape[1]

In [11]:
def unpreprocess(x):
    x = np.copy(x)
    # undo preprocesing
    mean = [103.939, 116.779, 123.68]
    x[:,:,0] += mean[0]
    x[:,:,1] += mean[1]
    x[:,:,2] += mean[2]
    x = x[..., ::-1]
    return x.astype(int)

In [12]:
averaged_X = input_imgs.mean(axis=0)

In [13]:
random_X = np.random.uniform(0, 255, (input_shape[1], input_shape[2], input_shape[3]))
random_X = preprocess_input(random_X)

In [14]:
cache = CacheStorage()
print("Try to load cache file")
predictions = cache.get_cache(os.path.join('cache', "imagenet_predictions"))
if predictions is None:
    print("Making cache file")
    predictions = model.predict(input_imgs)
    cache.set_cache(os.path.join('cache', "imagenet_predictions"), predictions)

Try to load cache file
Making cache file


In [15]:
from utils.visualizations import GradCAM, GuidedGradCAM, GBP, LRP, CLRP, SGLRP, SGLRP2
from tqdm import tqdm

In [16]:
from utils.visualizations_working import LRPA, IG, SG, IxG, Decon, LRPA, LRPB, LRPE, LRPA2B1, DL

In [17]:
def define_mask(row, col):
#         print(row, col)
    top = row - 9 // 2
    bottom = row + 9 // 2 + 1
    left = col - 9 // 2
    right = col + 9 // 2 + 1

    if top < 0:
        top = 0
    if bottom > input_shape[1] - 1:
        bottom = input_shape[1] - 1
    if left < 0:
        left = 0
    if right > input_shape[2] - 1:
        right = input_shape[2] - 1
    return top, bottom, left, right

In [18]:
num_iters = 100

temp_limit = 50000

gt = np.argmax(val_labels[:temp_limit], axis=1)


In [19]:
   
def show_image(img):
    plt.imshow(unpreprocess(img))
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.show()

In [20]:
verbose = 0
random_or_average = "random"

In [21]:
store_dir = os.path.join("maps", "imagenet")

In [23]:
import itertools
def get_maps(network, method):
    npz_path_list = [os.path.join(store_dir, "%d_%d_%s_%s.npz"%(c, i, network, method)) for c, i in itertools.product(range(1000), range(50))]
    saliency_maps = np.array([np.load(npz_path)["x"] for npz_path in tqdm(npz_path_list)])
#     print(npz_path_list)
    return saliency_maps
# get_maps("vgg19", "lrp")

In [21]:
# orig_maps = run_lrp(partial_model, f, input_imgs[:temp_limit])

In [26]:
print("Try to load cache file")
orig_maps = cache.get_cache(os.path.join('cache', "lrp_maps_gt_imagenet"))
if orig_maps is None:
    print("Making cache file")
    orig_maps = get_maps("vgg19", "lrpa")
    cache.set_cache(os.path.join('cache', "lrp_maps_gt_imagenet"), orig_maps)



  0%|          | 0/50000 [00:00<?, ?it/s]

Try to load cache file
Making cache file




  0%|          | 39/50000 [00:00<02:11, 381.30it/s]

  0%|          | 78/50000 [00:00<02:10, 381.78it/s]

  0%|          | 112/50000 [00:00<02:16, 366.18it/s]

  0%|          | 154/50000 [00:00<02:11, 379.69it/s]

  0%|          | 190/50000 [00:00<02:13, 372.68it/s]

  0%|          | 228/50000 [00:00<02:13, 373.79it/s]

  1%|          | 266/50000 [00:00<02:12, 375.15it/s]

  1%|          | 304/50000 [00:00<02:12, 375.72it/s]

  1%|          | 342/50000 [00:00<02:12, 376.18it/s]

  1%|          | 381/50000 [00:01<02:11, 378.33it/s]

  1%|          | 420/50000 [00:01<02:11, 376.16it/s]

  1%|          | 460/50000 [00:01<02:09, 382.16it/s]

  1%|          | 505/50000 [00:01<02:04, 398.57it/s]

  1%|          | 553/50000 [00:01<01:57, 419.75it/s]

  1%|          | 602/50000 [00:01<01:53, 436.70it/s]

  1%|▏         | 651/50000 [00:01<01:49, 449.44it/s]

  1%|▏         | 700/50000 [00:01<01:47, 458.60it/s]

  1%|▏         | 747/50000 [00:01<01:49, 449.35it/s]

  2%|▏         | 796/50000 [

 13%|█▎        | 6529/50000 [00:13<01:27, 494.91it/s]

 13%|█▎        | 6579/50000 [00:13<01:27, 496.14it/s]

 13%|█▎        | 6629/50000 [00:13<01:28, 491.74it/s]

 13%|█▎        | 6680/50000 [00:14<01:27, 494.55it/s]

 13%|█▎        | 6730/50000 [00:14<01:27, 496.12it/s]

 14%|█▎        | 6780/50000 [00:14<01:26, 496.89it/s]

 14%|█▎        | 6830/50000 [00:14<01:26, 497.49it/s]

 14%|█▍        | 6880/50000 [00:14<01:27, 493.44it/s]

 14%|█▍        | 6930/50000 [00:14<01:27, 494.42it/s]

 14%|█▍        | 6980/50000 [00:14<01:26, 495.73it/s]

 14%|█▍        | 7030/50000 [00:14<01:27, 492.17it/s]

 14%|█▍        | 7080/50000 [00:14<01:28, 485.31it/s]

 14%|█▍        | 7130/50000 [00:14<01:27, 489.23it/s]

 14%|█▍        | 7181/50000 [00:15<01:26, 492.72it/s]

 14%|█▍        | 7232/50000 [00:15<01:26, 495.19it/s]

 15%|█▍        | 7282/50000 [00:15<01:26, 491.99it/s]

 15%|█▍        | 7332/50000 [00:15<01:26, 493.91it/s]

 15%|█▍        | 7383/50000 [00:15<01:25, 496.32it/s]

 15%|█▍   

 26%|██▋       | 13206/50000 [00:27<01:14, 496.77it/s]

 27%|██▋       | 13256/50000 [00:27<01:13, 496.94it/s]

 27%|██▋       | 13306/50000 [00:27<01:14, 490.99it/s]

 27%|██▋       | 13356/50000 [00:27<01:14, 493.00it/s]

 27%|██▋       | 13406/50000 [00:27<01:13, 494.59it/s]

 27%|██▋       | 13457/50000 [00:27<01:13, 496.25it/s]

 27%|██▋       | 13508/50000 [00:27<01:13, 497.71it/s]

 27%|██▋       | 13558/50000 [00:27<01:13, 492.58it/s]

 27%|██▋       | 13608/50000 [00:28<01:13, 494.29it/s]

 27%|██▋       | 13658/50000 [00:28<01:13, 495.96it/s]

 27%|██▋       | 13709/50000 [00:28<01:12, 497.27it/s]

 28%|██▊       | 13759/50000 [00:28<01:12, 497.68it/s]

 28%|██▊       | 13809/50000 [00:28<01:16, 475.94it/s]

 28%|██▊       | 13859/50000 [00:28<01:14, 482.19it/s]

 28%|██▊       | 13909/50000 [00:28<01:14, 486.81it/s]

 28%|██▊       | 13959/50000 [00:28<01:13, 489.91it/s]

 28%|██▊       | 14009/50000 [00:28<01:13, 488.01it/s]

 28%|██▊       | 14058/50000 [00:28<01:14, 482.6

 40%|███▉      | 19788/50000 [00:40<01:00, 498.10it/s]

 40%|███▉      | 19838/50000 [00:40<01:00, 496.10it/s]

 40%|███▉      | 19888/50000 [00:40<01:00, 496.88it/s]

 40%|███▉      | 19938/50000 [00:41<01:00, 497.75it/s]

 40%|███▉      | 19988/50000 [00:41<01:00, 497.59it/s]

 40%|████      | 20039/50000 [00:41<01:00, 498.41it/s]

 40%|████      | 20089/50000 [00:41<01:00, 495.16it/s]

 40%|████      | 20139/50000 [00:41<01:00, 496.02it/s]

 40%|████      | 20189/50000 [00:41<01:00, 496.66it/s]

 40%|████      | 20239/50000 [00:41<00:59, 497.03it/s]

 41%|████      | 20289/50000 [00:41<00:59, 497.07it/s]

 41%|████      | 20339/50000 [00:41<01:00, 492.95it/s]

 41%|████      | 20390/50000 [00:41<00:59, 495.22it/s]

 41%|████      | 20441/50000 [00:42<00:59, 496.87it/s]

 41%|████      | 20491/50000 [00:42<00:59, 497.31it/s]

 41%|████      | 20541/50000 [00:42<00:59, 494.06it/s]

 41%|████      | 20591/50000 [00:42<00:59, 495.61it/s]

 41%|████▏     | 20642/50000 [00:42<00:59, 497.0

 53%|█████▎    | 26412/50000 [00:54<00:48, 486.38it/s]

 53%|█████▎    | 26462/50000 [00:54<00:48, 489.45it/s]

 53%|█████▎    | 26512/50000 [00:54<00:47, 491.94it/s]

 53%|█████▎    | 26562/50000 [00:54<00:47, 493.55it/s]

 53%|█████▎    | 26612/50000 [00:54<00:47, 494.80it/s]

 53%|█████▎    | 26662/50000 [00:54<00:47, 495.57it/s]

 53%|█████▎    | 26712/50000 [00:54<00:50, 462.63it/s]

 54%|█████▎    | 26759/50000 [00:54<00:52, 445.85it/s]

 54%|█████▎    | 26809/50000 [00:54<00:50, 459.70it/s]

 54%|█████▎    | 26859/50000 [00:55<00:49, 470.97it/s]

 54%|█████▍    | 26909/50000 [00:55<00:48, 478.42it/s]

 54%|█████▍    | 26959/50000 [00:55<00:47, 484.34it/s]

 54%|█████▍    | 27009/50000 [00:55<00:47, 488.49it/s]

 54%|█████▍    | 27059/50000 [00:55<00:46, 490.68it/s]

 54%|█████▍    | 27109/50000 [00:55<00:46, 492.84it/s]

 54%|█████▍    | 27159/50000 [00:55<00:46, 494.18it/s]

 54%|█████▍    | 27209/50000 [00:55<00:46, 495.34it/s]

 55%|█████▍    | 27259/50000 [00:55<00:48, 473.4

 66%|██████▌   | 32949/50000 [01:07<00:36, 467.37it/s]

 66%|██████▌   | 32997/50000 [01:07<00:36, 468.98it/s]

 66%|██████▌   | 33045/50000 [01:07<00:36, 469.99it/s]

 66%|██████▌   | 33093/50000 [01:07<00:35, 470.29it/s]

 66%|██████▋   | 33141/50000 [01:08<00:35, 468.49it/s]

 66%|██████▋   | 33189/50000 [01:08<00:35, 469.70it/s]

 66%|██████▋   | 33237/50000 [01:08<00:35, 470.34it/s]

 67%|██████▋   | 33285/50000 [01:08<00:35, 470.68it/s]

 67%|██████▋   | 33333/50000 [01:08<00:35, 469.27it/s]

 67%|██████▋   | 33381/50000 [01:08<00:35, 470.20it/s]

 67%|██████▋   | 33429/50000 [01:08<00:35, 470.60it/s]

 67%|██████▋   | 33477/50000 [01:08<00:35, 470.80it/s]

 67%|██████▋   | 33525/50000 [01:08<00:34, 471.28it/s]

 67%|██████▋   | 33573/50000 [01:08<00:35, 469.07it/s]

 67%|██████▋   | 33620/50000 [01:09<00:34, 469.19it/s]

 67%|██████▋   | 33668/50000 [01:09<00:34, 470.12it/s]

 67%|██████▋   | 33716/50000 [01:09<00:34, 470.91it/s]

 68%|██████▊   | 33764/50000 [01:09<00:34, 470.7

 79%|███████▉  | 39518/50000 [01:21<00:21, 499.05it/s]

 79%|███████▉  | 39568/50000 [01:21<00:20, 498.66it/s]

 79%|███████▉  | 39618/50000 [01:21<00:20, 496.78it/s]

 79%|███████▉  | 39668/50000 [01:21<00:20, 497.48it/s]

 79%|███████▉  | 39719/50000 [01:21<00:20, 498.46it/s]

 80%|███████▉  | 39770/50000 [01:21<00:20, 499.43it/s]

 80%|███████▉  | 39821/50000 [01:21<00:20, 499.82it/s]

 80%|███████▉  | 39871/50000 [01:21<00:20, 498.30it/s]

 80%|███████▉  | 39922/50000 [01:21<00:20, 498.88it/s]

 80%|███████▉  | 39973/50000 [01:21<00:20, 499.56it/s]

 80%|████████  | 40024/50000 [01:22<00:19, 499.95it/s]

 80%|████████  | 40074/50000 [01:22<00:19, 498.51it/s]

 80%|████████  | 40125/50000 [01:22<00:19, 499.15it/s]

 80%|████████  | 40175/50000 [01:22<00:19, 498.89it/s]

 80%|████████  | 40225/50000 [01:22<00:19, 498.93it/s]

 81%|████████  | 40275/50000 [01:22<00:19, 495.41it/s]

 81%|████████  | 40325/50000 [01:22<00:19, 496.60it/s]

 81%|████████  | 40376/50000 [01:22<00:19, 497.7

 92%|█████████▏| 46181/50000 [01:34<00:07, 481.69it/s]

 92%|█████████▏| 46231/50000 [01:34<00:07, 486.84it/s]

 93%|█████████▎| 46282/50000 [01:34<00:07, 490.78it/s]

 93%|█████████▎| 46333/50000 [01:34<00:07, 493.75it/s]

 93%|█████████▎| 46383/50000 [01:35<00:07, 495.49it/s]

 93%|█████████▎| 46434/50000 [01:35<00:07, 497.20it/s]

 93%|█████████▎| 46485/50000 [01:35<00:07, 498.18it/s]

 93%|█████████▎| 46536/50000 [01:35<00:06, 499.04it/s]

 93%|█████████▎| 46586/50000 [01:35<00:06, 498.23it/s]

 93%|█████████▎| 46636/50000 [01:35<00:06, 497.81it/s]

 93%|█████████▎| 46687/50000 [01:35<00:06, 498.90it/s]

 93%|█████████▎| 46737/50000 [01:35<00:06, 498.33it/s]

 94%|█████████▎| 46787/50000 [01:35<00:06, 498.77it/s]

 94%|█████████▎| 46837/50000 [01:35<00:06, 498.49it/s]

 94%|█████████▍| 46887/50000 [01:36<00:06, 497.40it/s]

 94%|█████████▍| 46937/50000 [01:36<00:06, 498.06it/s]

 94%|█████████▍| 46987/50000 [01:36<00:06, 498.54it/s]

 94%|█████████▍| 47037/50000 [01:36<00:05, 498.1

In [27]:
def run_change_in_y_t(model, targets, images, maps, num_iters, random_X, averaged_X):
    input_shape = images.shape
    ret = np.zeros((input_shape[0], num_iters))

    for example_id, target_id in enumerate(tqdm(targets)):
        modified_images = np.zeros((num_iters, input_shape[1], input_shape[2], input_shape[3]))
        modified_images[0] = np.copy(images[example_id])
        current_map = np.copy(maps[example_id])
        for i in np.arange(1,num_iters):
            row, col = np.unravel_index(current_map.argmax(), current_map.shape)

            # wipe out used
            current_map[row, col] = 0

            top, bottom, left, right = define_mask(row, col)
            modified_images[i] = modified_images[i-1]
            if random_or_average == "random":
                modified_images[i, top:bottom, left:right] = random_X[top:bottom, left:right]
            else:
                modified_images[i, top:bottom, left:right] = averaged_X[top:bottom, left:right]
#             if i == 99:
#                 print(top, bottom, left, right)
#                 show_image(modified_images[i])
        ret[example_id] = model.predict(modified_images)[:,target_id]
    return ret

In [ ]:
results = run_change_in_y_t(model, gt, input_imgs[:temp_limit], orig_maps, num_iters, random_X, averaged_X)



  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 1/50000 [00:08<112:47:24,  8.12s/it]

  0%|          | 2/50000 [00:15<109:41:43,  7.90s/it]

  0%|          | 3/50000 [00:22<107:55:22,  7.77s/it]

  0%|          | 4/50000 [00:30<106:04:06,  7.64s/it]

  0%|          | 5/50000 [00:37<104:54:42,  7.55s/it]

  0%|          | 6/50000 [00:45<104:07:36,  7.50s/it]

  0%|          | 7/50000 [00:52<103:42:18,  7.47s/it]

  0%|          | 8/50000 [00:59<103:41:02,  7.47s/it]

  0%|          | 9/50000 [01:07<103:21:00,  7.44s/it]

  0%|          | 10/50000 [01:14<102:35:46,  7.39s/it]

  0%|          | 11/50000 [01:21<102:37:58,  7.39s/it]

  0%|          | 12/50000 [01:29<102:34:36,  7.39s/it]

  0%|          | 13/50000 [01:36<102:12:57,  7.36s/it]

  0%|          | 14/50000 [01:43<101:45:53,  7.33s/it]

  0%|          | 15/50000 [01:51<101:41:53,  7.32s/it]

  0%|          | 16/50000 [01:58<101:46:01,  7.33s/it]

  0%|          | 17/50000 [02:05<102:07:59,  7.36s/it]

  0%|       

  0%|          | 131/50000 [16:16<103:35:19,  7.48s/it]

  0%|          | 132/50000 [16:23<103:19:42,  7.46s/it]

  0%|          | 133/50000 [16:31<103:00:51,  7.44s/it]

  0%|          | 134/50000 [16:38<102:56:10,  7.43s/it]

  0%|          | 135/50000 [16:46<102:44:30,  7.42s/it]

  0%|          | 136/50000 [16:53<102:51:20,  7.43s/it]

  0%|          | 137/50000 [17:01<103:20:02,  7.46s/it]

  0%|          | 138/50000 [17:08<103:32:48,  7.48s/it]

  0%|          | 139/50000 [17:16<103:24:40,  7.47s/it]

  0%|          | 140/50000 [17:23<103:12:31,  7.45s/it]

  0%|          | 141/50000 [17:30<103:08:51,  7.45s/it]

  0%|          | 142/50000 [17:38<103:12:44,  7.45s/it]

  0%|          | 143/50000 [17:45<103:03:38,  7.44s/it]

  0%|          | 144/50000 [17:53<103:18:00,  7.46s/it]

  0%|          | 145/50000 [18:00<103:12:18,  7.45s/it]

  0%|          | 146/50000 [18:08<103:16:17,  7.46s/it]

  0%|          | 147/50000 [18:15<103:13:46,  7.45s/it]

  0%|          | 148/50000 [18:

  1%|          | 261/50000 [32:22<101:26:59,  7.34s/it]

  1%|          | 262/50000 [32:30<101:27:19,  7.34s/it]

  1%|          | 263/50000 [32:37<101:42:28,  7.36s/it]

  1%|          | 264/50000 [32:44<101:43:33,  7.36s/it]

  1%|          | 265/50000 [32:52<101:39:51,  7.36s/it]

  1%|          | 266/50000 [32:59<101:45:34,  7.37s/it]

  1%|          | 267/50000 [33:06<101:39:31,  7.36s/it]

  1%|          | 268/50000 [33:14<101:59:39,  7.38s/it]

  1%|          | 269/50000 [33:21<102:07:13,  7.39s/it]

  1%|          | 270/50000 [33:29<102:04:34,  7.39s/it]

  1%|          | 271/50000 [33:36<101:34:00,  7.35s/it]

  1%|          | 272/50000 [33:43<102:10:19,  7.40s/it]

  1%|          | 273/50000 [33:51<102:01:45,  7.39s/it]

  1%|          | 274/50000 [33:58<101:41:01,  7.36s/it]

  1%|          | 275/50000 [34:05<101:37:04,  7.36s/it]

  1%|          | 276/50000 [34:13<101:30:00,  7.35s/it]

  1%|          | 277/50000 [34:20<101:21:43,  7.34s/it]

  1%|          | 278/50000 [34:

  1%|          | 391/50000 [48:15<100:36:56,  7.30s/it]

  1%|          | 392/50000 [48:23<100:18:39,  7.28s/it]

  1%|          | 393/50000 [48:30<100:27:44,  7.29s/it]

  1%|          | 394/50000 [48:37<100:38:02,  7.30s/it]

  1%|          | 395/50000 [48:45<100:39:27,  7.31s/it]

  1%|          | 396/50000 [48:52<100:26:31,  7.29s/it]

  1%|          | 397/50000 [48:59<100:30:32,  7.29s/it]

  1%|          | 398/50000 [49:06<100:28:47,  7.29s/it]

  1%|          | 399/50000 [49:14<100:08:03,  7.27s/it]

  1%|          | 400/50000 [49:21<100:03:39,  7.26s/it]

  1%|          | 401/50000 [49:28<99:54:40,  7.25s/it] 

  1%|          | 402/50000 [49:35<99:58:42,  7.26s/it]

  1%|          | 403/50000 [49:43<99:52:35,  7.25s/it]

  1%|          | 404/50000 [49:50<100:00:31,  7.26s/it]

  1%|          | 405/50000 [49:57<100:07:47,  7.27s/it]

  1%|          | 406/50000 [50:04<100:04:32,  7.26s/it]

  1%|          | 407/50000 [50:12<100:18:39,  7.28s/it]

  1%|          | 408/50000 [50:19

  1%|          | 521/50000 [1:03:56<98:53:31,  7.20s/it]

  1%|          | 522/50000 [1:04:03<98:58:45,  7.20s/it]

  1%|          | 523/50000 [1:04:11<99:07:35,  7.21s/it]

  1%|          | 524/50000 [1:04:18<99:06:21,  7.21s/it]

  1%|          | 525/50000 [1:04:25<99:08:47,  7.21s/it]

  1%|          | 526/50000 [1:04:32<99:05:27,  7.21s/it]

  1%|          | 527/50000 [1:04:39<98:37:47,  7.18s/it]

  1%|          | 528/50000 [1:04:47<98:26:46,  7.16s/it]

  1%|          | 529/50000 [1:04:54<98:25:14,  7.16s/it]

  1%|          | 530/50000 [1:05:01<98:32:08,  7.17s/it]

  1%|          | 531/50000 [1:05:08<98:31:00,  7.17s/it]

  1%|          | 532/50000 [1:05:15<98:59:18,  7.20s/it]

  1%|          | 533/50000 [1:05:23<99:02:29,  7.21s/it]

  1%|          | 534/50000 [1:05:30<98:56:39,  7.20s/it]

  1%|          | 535/50000 [1:05:37<100:00:30,  7.28s/it]

  1%|          | 536/50000 [1:05:45<100:45:07,  7.33s/it]

  1%|          | 537/50000 [1:05:52<100:05:27,  7.28s/it]

  1%|      

...

 18%|█▊        | 9017/50000 [18:32:00<132:49:30, 11.67s/it]

 18%|█▊        | 9018/50000 [18:32:11<133:24:15, 11.72s/it]

 18%|█▊        | 9019/50000 [18:32:24<134:38:02, 11.83s/it]

 18%|█▊        | 9020/50000 [18:32:35<134:05:24, 11.78s/it]

 18%|█▊        | 9021/50000 [18:32:47<133:29:09, 11.73s/it]

 18%|█▊        | 9022/50000 [18:32:59<134:40:40, 11.83s/it]

 18%|█▊        | 9023/50000 [18:33:11<134:41:31, 11.83s/it]

 18%|█▊        | 9024/50000 [18:33:22<134:09:49, 11.79s/it]

 18%|█▊        | 9025/50000 [18:33:34<134:12:32, 11.79s/it]

 18%|█▊        | 9026/50000 [18:33:46<134:02:15, 11.78s/it]

 18%|█▊        | 9027/50000 [18:33:58<134:15:35, 11.80s/it]

 18%|█▊        | 9028/50000 [18:34:10<134:22:09, 11.81s/it]

 18%|█▊        | 9029/50000 [18:34:22<134:53:59, 11.85s/it]

 18%|█▊        | 9030/50000 [18:34:33<134:36:36, 11.83s/it]

 18%|█▊        | 9031/50000 [18:34:45<134:12:48, 11.79s/it]

 18%|█▊        | 9032/50000 [18:34:57<134:53:17, 11.85s/it]

 18%|█▊        | 9033/50

 18%|█▊        | 9139/50000 [18:56:07<140:41:00, 12.39s/it]

 18%|█▊        | 9140/50000 [18:56:20<140:11:31, 12.35s/it]

 18%|█▊        | 9141/50000 [18:56:32<138:50:34, 12.23s/it]

 18%|█▊        | 9142/50000 [18:56:44<140:42:01, 12.40s/it]

 18%|█▊        | 9143/50000 [18:56:57<140:46:56, 12.40s/it]

 18%|█▊        | 9144/50000 [18:57:09<141:23:57, 12.46s/it]

 18%|█▊        | 9145/50000 [18:57:22<140:48:50, 12.41s/it]

 18%|█▊        | 9146/50000 [18:57:34<139:39:01, 12.31s/it]

 18%|█▊        | 9147/50000 [18:57:46<139:35:49, 12.30s/it]

 18%|█▊        | 9148/50000 [18:57:58<138:59:17, 12.25s/it]

 18%|█▊        | 9149/50000 [18:58:11<139:20:32, 12.28s/it]

 18%|█▊        | 9150/50000 [18:58:23<138:35:37, 12.21s/it]

 18%|█▊        | 9151/50000 [18:58:35<138:43:50, 12.23s/it]

 18%|█▊        | 9152/50000 [18:58:47<139:09:43, 12.26s/it]

 18%|█▊        | 9153/50000 [18:59:00<141:15:35, 12.45s/it]

 18%|█▊        | 9154/50000 [18:59:13<142:16:31, 12.54s/it]

 18%|█▊        | 9155/50

 19%|█▊        | 9261/50000 [19:21:49<141:13:49, 12.48s/it]

 19%|█▊        | 9262/50000 [19:22:02<142:52:20, 12.63s/it]

 19%|█▊        | 9263/50000 [19:22:14<141:07:30, 12.47s/it]

 19%|█▊        | 9264/50000 [19:22:27<142:31:37, 12.60s/it]

 19%|█▊        | 9265/50000 [19:22:39<142:02:11, 12.55s/it]

 19%|█▊        | 9266/50000 [19:22:52<143:32:16, 12.69s/it]

 19%|█▊        | 9267/50000 [19:23:04<142:19:10, 12.58s/it]

 19%|█▊        | 9268/50000 [19:23:17<140:58:17, 12.46s/it]

 19%|█▊        | 9269/50000 [19:23:29<140:53:41, 12.45s/it]

 19%|█▊        | 9270/50000 [19:23:41<140:43:24, 12.44s/it]

 19%|█▊        | 9271/50000 [19:23:53<139:23:29, 12.32s/it]

 19%|█▊        | 9272/50000 [19:24:06<139:48:12, 12.36s/it]

 19%|█▊        | 9273/50000 [19:24:18<140:21:04, 12.41s/it]

 19%|█▊        | 9274/50000 [19:24:31<139:53:58, 12.37s/it]

 19%|█▊        | 9275/50000 [19:24:44<142:21:00, 12.58s/it]

 19%|█▊        | 9276/50000 [19:24:56<142:20:01, 12.58s/it]

 19%|█▊        | 9277/50

 19%|█▉        | 9383/50000 [19:47:18<139:17:46, 12.35s/it]

 19%|█▉        | 9384/50000 [19:47:30<138:22:46, 12.27s/it]

 19%|█▉        | 9385/50000 [19:47:43<139:58:04, 12.41s/it]

 19%|█▉        | 9386/50000 [19:47:56<141:07:14, 12.51s/it]

 19%|█▉        | 9387/50000 [19:48:08<140:21:31, 12.44s/it]

 19%|█▉        | 9388/50000 [19:48:16<124:15:42, 11.02s/it]

 19%|█▉        | 9389/50000 [19:48:23<112:11:19,  9.95s/it]

 19%|█▉        | 9390/50000 [19:48:31<103:58:00,  9.22s/it]

 19%|█▉        | 9391/50000 [19:48:38<98:11:51,  8.71s/it] 

 19%|█▉        | 9392/50000 [19:48:46<94:06:39,  8.34s/it]

 19%|█▉        | 9393/50000 [19:48:53<90:56:19,  8.06s/it]

 19%|█▉        | 9394/50000 [19:49:01<89:04:01,  7.90s/it]

 19%|█▉        | 9395/50000 [19:49:08<87:41:53,  7.78s/it]

 19%|█▉        | 9396/50000 [19:49:16<86:47:47,  7.70s/it]

 19%|█▉        | 9397/50000 [19:49:23<85:58:09,  7.62s/it]

 19%|█▉        | 9398/50000 [19:49:31<85:30:45,  7.58s/it]

 19%|█▉        | 9399/50000 [19

 19%|█▉        | 9507/50000 [20:03:06<83:59:13,  7.47s/it]

 19%|█▉        | 9508/50000 [20:03:13<83:54:46,  7.46s/it]

 19%|█▉        | 9509/50000 [20:03:21<83:50:25,  7.45s/it]

 19%|█▉        | 9510/50000 [20:03:28<84:00:59,  7.47s/it]

 19%|█▉        | 9511/50000 [20:03:36<84:34:07,  7.52s/it]

 19%|█▉        | 9512/50000 [20:03:43<84:13:24,  7.49s/it]

 19%|█▉        | 9513/50000 [20:03:51<84:05:46,  7.48s/it]

 19%|█▉        | 9514/50000 [20:03:58<84:11:28,  7.49s/it]

 19%|█▉        | 9515/50000 [20:04:06<84:13:24,  7.49s/it]

 19%|█▉        | 9516/50000 [20:04:13<84:10:26,  7.49s/it]

In [ ]:
# print("Try to load cache file")
# results = cache.get_cache(os.path.join('cache', "change_yt_lrp_gt_imagenet"))
# if results is None:
#     print("Making cache file")
#     results = run_change_in_y_t(model, gt, input_imgs[:temp_limit], orig_maps, num_iters, random_X, averaged_X)
#     cache.set_cache(os.path.join('cache', "change_yt_lrp_gt_imagenet"), results)

In [ ]:
ave_results = np.mean(results, axis=0)

plt.plot(np.arange(num_iters), ave_results)
plt.show()


In [ ]:
#aopc
aopc = 1./(num_iters+1.)*np.cumsum(ave_results)
plt.plot(np.arange(num_iters), aopc)
plt.show()

In [ ]:
heatmap(orig_maps[0])
plt.show()